# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tianfu,34.2650,108.5904,274.68,43,5,2.91,CN,"time.struct_time(tm_year=2025, tm_mon=2, tm_md..."
1,1,kingston,17.9712,-76.7928,300.05,78,75,6.69,JM,"time.struct_time(tm_year=2025, tm_mon=2, tm_md..."
2,2,olonkinbyen,70.9224,-8.7160,271.93,83,100,8.73,SJ,"time.struct_time(tm_year=2025, tm_mon=2, tm_md..."
3,3,invercargill,-46.4118,168.3471,281.49,94,38,1.64,NZ,"time.struct_time(tm_year=2025, tm_mon=2, tm_md..."
4,4,vilyuchinsk,52.9318,158.4054,265.32,93,96,1.98,RU,"time.struct_time(tm_year=2025, tm_mon=2, tm_md..."


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
mapplot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    color = 'City',
    size = 'Humidity'
)
# Display the map
# YOUR CODE HERE
mapplot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
city_data_df['tempc'] = city_data_df['Max Temp'] - 273.15

# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df['tempc'] < 20) & (city_data_df['tempc'] > 10)]
city_data_df = city_data_df[city_data_df['Cloudiness'] > 10]
# Drop any rows with null values


# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,tempc
11,11,puerto deseado,-47.7516,-65.9013,288.48,45,13,8.43,AR,"time.struct_time(tm_year=2025, tm_mon=2, tm_md...",15.33
12,12,moab,38.5738,-109.5462,292.32,23,100,2.24,US,"time.struct_time(tm_year=2025, tm_mon=2, tm_md...",19.17
16,16,port lincoln,-34.7212,135.8592,290.79,68,36,7.91,AU,"time.struct_time(tm_year=2025, tm_mon=2, tm_md...",17.64
17,17,fale old settlement,-34.7212,135.8592,290.79,68,36,7.91,AU,"time.struct_time(tm_year=2025, tm_mon=2, tm_md...",17.64
20,20,edinburgh of the seven seas,-37.0676,-12.3108,288.45,70,19,5.52,SH,"time.struct_time(tm_year=2025, tm_mon=2, tm_md...",15.30


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
11,puerto deseado,AR,-47.7516,-65.9013,45,None
12,moab,US,38.5738,-109.5462,23,None
16,port lincoln,AU,-34.7212,135.8592,68,None
17,fale old settlement,AU,-34.7212,135.8592,68,None
20,edinburgh of the seven seas,SH,-37.0676,-12.3108,70,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = # YOUR CODE HERE

    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
badger - nearest hotel: Clarion Hotel & Suites
ust'-kulom - nearest hotel: No hotel found
emerald - nearest hotel: Emerald Central Hotel
newman - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
laguna - nearest hotel: Holiday Inn Express & Suites
nova vicosa - nearest hotel: Pousada Pontal da Barra
fort myers shores - nearest hotel: Comfort Inn Fort Myers
trinity beach - nearest hotel: Blue Lagoon Resort
racine - nearest hotel: No hotel found
six-fours-les-plages - nearest hotel: Kyriad Direct La Seyne-sur-Mer
porto velho - nearest hotel: Hotel Slaviero Essential Porto Velho
posto fiscal rolim de moura - nearest hotel: No hotel found
port douglas - nearest hotel: Mantra Aqueous Port Douglas
roswell - nearest hotel: Best Western Plus Hotel
hell-ville - nearest hotel: ABUD HOTEL
sona - nearest hotel: Al Bosco
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by W

,City,Country,Lat,Lng,Humidity,Hotel Name
43,saint-pierre,RE,-21.3393,55.4781,61,Tropic Hotel
66,badger,US,64.8000,-147.5333,37,Clarion Hotel & Suites
102,ust'-kulom,RU,61.6864,53.6902,43,No hotel found
120,emerald,AU,-23.5333,148.1667,28,Emerald Central Hotel
130,newman,US,37.3138,-121.0208,51,No hotel found
138,fort bragg,US,35.1390,-79.0060,80,Airborne Inn Lodging
146,laguna,US,38.4210,-121.4238,52,Holiday Inn Express & Suites
160,nova vicosa,BR,-17.8919,-39.3719,87,Pousada Pontal da Barra
170,fort myers shores,US,26.7092,-81.7459,95,Comfort Inn Fort Myers
176,trinity beach,AU,-16.7888,145.6968,44,Blue Lagoon Resort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)